## Analysis of HUMAN Methylation Data from the Mammalian Methylation Consortium dataset (Horvath 2023)

need to update the title and description of this notebook as serves mainly to clean and export methylation data (complete, human) and Clock data (complete & human) 

In [1]:
# Import 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import re
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy import stats
import statsmodels.stats.multitest as multi
from fastcluster import linkage
import plotly.graph_objects as go
import pandas as pd


### 1. Data Cleaning

#### 1.1 Import the datasets (Complete & Human) from the Mammalian Methylation Consortium dataset (Horvath 2023) 

In [2]:
# Read the Human CSV file
human_methylation_data = pd.read_csv('GSE184211_datBetaNormalized.csv')

In [3]:
# Read the Complete CSV file
complete_methylation_data = pd.read_csv('GSE223748_datBetaNormalized.csv')

In [14]:
print(human_methylation_data.shape)
print(complete_methylation_data.shape)

(37554, 662)
(37554, 15044)


#### 1.2. Rename the columns of the datasets to make the datasets more readable and standardized

In [5]:
# Rename column 'CGid' to 'CpG'
human_methylation_data.rename(columns={'CGid': 'CpG'}, inplace=True)
complete_methylation_data.rename(columns={'Unnamed: 0': 'CpG'}, inplace=True)

In [6]:
complete_methylation_data.head(5)

,CpG,202897220093_R01C01,202897220093_R02C01,202897220093_R03C01,202897220093_R04C01,202897220093_R05C01,202897220093_R06C01,202897220093_R01C02,202897220093_R02C02,202897220093_R03C02,...,206139140103_R02C02,206139140103_R03C02,206139140103_R04C02,206139140103_R05C02,206139140103_R06C02,206139140104_R01C01,206139140104_R02C01,206139140104_R03C01,206139140104_R04C01,206139140104_R05C01
0,cg00000165,0.480990,0.483132,0.494042,0.505628,0.496484,0.486760,0.497341,0.482591,0.495475,...,0.140734,0.070563,0.060902,0.056785,0.054896,0.218780,0.141689,0.092939,0.085170,0.060552
1,cg00001209,0.934681,0.946624,0.940368,0.935963,0.926762,0.931264,0.941756,0.936942,0.940002,...,0.833588,0.872219,0.900707,0.929456,0.861758,0.828209,0.868502,0.825405,0.907443,0.909347
2,cg00001364,0.911639,0.907298,0.912401,0.901470,0.912350,0.910808,0.911290,0.910110,0.907811,...,0.808296,0.858634,0.840960,0.903902,0.914616,0.778192,0.805107,0.832695,0.889494,0.901459
3,cg00001582,0.060784,0.057917,0.055478,0.064311,0.060102,0.055498,0.056630,0.060347,0.066302,...,0.077506,0.087474,0.081258,0.067971,0.067423,0.069621,0.075440,0.065853,0.074909,0.074081
4,cg00002920,0.655778,0.628036,0.697029,0.690823,0.678101,0.615883,0.638421,0.641940,0.654358,...,0.719434,0.782699,0.701002,0.638727,0.755281,0.776376,0.799998,0.781141,0.698296,0.731993


#### 1.3. Check and handle missing values

In [7]:
# Check if there are any missing values
complete_missing_values = complete_methylation_data.isnull().sum().sum()
human_missing_values = human_methylation_data.isnull().sum().sum()
print(f'There are {complete_missing_values} missing values in the complete methylation dataset.')
print(f'There are {human_missing_values} missing values in the human methylation dataset.')

There are 0 missing values in the complete methylation dataset.
There are 0 missing values in the human methylation dataset.


#### 1.4. Check and handle duplicates

In [8]:
# Check if there are any duplicated CpG sites
duplicated_cpg_complete = complete_methylation_data['CpG'].duplicated().sum()
duplicated_cpg_human = human_methylation_data['CpG'].duplicated().sum()
print(f'There are {duplicated_cpg_complete} duplicated CpG sites in the complete methylation dataset.')
print(f'There are {duplicated_cpg_human} duplicated CpG sites in the human methylation.')

There are 0 duplicated CpG sites in the complete methylation dataset.
There are 0 duplicated CpG sites in the human methylation.


### 2. Exploratory Analysis of the data with Heatmaps and PCA

2.1. Heatmap of the whole dataset

2.2. PCA of the whole dataset

### 3. Filtering the data by the CpGs identified in the Clocks & export clocks

#### 3.1. Import CpG's identified by the Clocks

In [9]:
# Import full clocks
Clock_1 = pd.read_csv('../FunctionalAnalysis/Clock_1_ordered.csv')
Clock_2 = pd.read_csv('../FunctionalAnalysis/Clock_2_ordered.csv')
Clock_3 = pd.read_csv('../FunctionalAnalysis/Clock_3_ordered.csv')
Clock_1_2_3_overlap = pd.read_csv('../FunctionalAnalysis/overlap_1_2_3_ordered.csv')
Clock_2_3_overlap = pd.read_csv('../FunctionalAnalysis/overlap_2_3_ordered.csv')

In [27]:
Clock_1.head()

,index,var,beta_clock1,CHR,bp_hg38,Gene,Gene.hg19,ENTREZID,conservationInMouse,GeneRegionID,annotation,CHR_mm10,bp_mm10,Gene_mm10
0,143,cg11728741,0.928874,8,41896469,ANK1,ANK1,286,conserved gene and region in mouse and human,ANK1_Exon,Promoter (<=1kb),8,22975146.0,Ank1
1,297,cg24352905,0.853268,5,77645452,OTP,OTP,23440,conserved gene and region in mouse and human,OTP_Promoter,Promoter (5-6kb),13,94869014.0,Otp
2,98,cg08938156,0.843749,3,147409417,LOC440982,ZIC1,440982,mapped to different gene in human and mouse,LOC440982_fiveUTR,5' UTR,9,91365746.0,Zic1
3,160,cg13058338,0.638660,12,54173598,SMUG1,SMUG1,23583,mapped to different gene in human and mouse,SMUG1_Intron,"Intron (ENST00000635234.1/23583, intron 2 of 4)",15,103146996.0,Smug1
4,316,cg26067250,0.510613,2,172085721,DLX1,DLX1,1745,mapped to different gene in human and mouse,DLX1_Exon,Promoter (<=1kb),2,71530037.0,Dlx1


In [28]:
Clock_1.shape

(335, 14)

#### 3.2. Create subsets of top CpGs identified by clocks

In [10]:
# Create a subset of the first 100 and 200 CpG sites for each clock

Clock1_100 = Clock_1.head(100)
Clock2_100 = Clock_2.head(100)
Clock3_100 = Clock_3.head(100)
Clock1_2_3_overlap_100 = Clock_1_2_3_overlap.head(100)
Clock2_3_overlap_100 = Clock_2_3_overlap.head(100)

Clock1_200 = Clock_1.head(200)
Clock2_200 = Clock_2.head(200)
Clock3_200 = Clock_3.head(200)
Clock1_2_3_overlap_200 = Clock_1_2_3_overlap.head(200)
Clock2_3_overlap_200 = Clock_2_3_overlap.head(200)


In [11]:
# Sneak peek of the top CpGs for clock 1
print(Clock1_100.shape)
Clock1_100.head()

(100, 14)


,index,var,beta_clock1,CHR,bp_hg38,Gene,Gene.hg19,ENTREZID,conservationInMouse,GeneRegionID,annotation,CHR_mm10,bp_mm10,Gene_mm10
0,143,cg11728741,0.928874,8,41896469,ANK1,ANK1,286,conserved gene and region in mouse and human,ANK1_Exon,Promoter (<=1kb),8,22975146.0,Ank1
1,297,cg24352905,0.853268,5,77645452,OTP,OTP,23440,conserved gene and region in mouse and human,OTP_Promoter,Promoter (5-6kb),13,94869014.0,Otp
2,98,cg08938156,0.843749,3,147409417,LOC440982,ZIC1,440982,mapped to different gene in human and mouse,LOC440982_fiveUTR,5' UTR,9,91365746.0,Zic1
3,160,cg13058338,0.638660,12,54173598,SMUG1,SMUG1,23583,mapped to different gene in human and mouse,SMUG1_Intron,"Intron (ENST00000635234.1/23583, intron 2 of 4)",15,103146996.0,Smug1
4,316,cg26067250,0.510613,2,172085721,DLX1,DLX1,1745,mapped to different gene in human and mouse,DLX1_Exon,Promoter (<=1kb),2,71530037.0,Dlx1


In [21]:
# Sneak peek of the overlaps (CpG col has a different name)
print(Clock1_2_3_overlap_100.shape)
Clock1_2_3_overlap_100.head()

(100, 40)


,index_x,var_x,beta_clock1,CHR_x,bp_hg38_x,Gene,Gene.hg19_x,ENTREZID_x,conservationInMouse_x,GeneRegionID_x,...,CHR,bp_hg38,Gene.hg19,ENTREZID,conservationInMouse,GeneRegionID,annotation,CHR_mm10,bp_mm10,Gene_mm10
0,98,cg08938156,0.843749,3,147409417,LOC440982,ZIC1,440982,mapped to different gene in human and mouse,LOC440982_fiveUTR,...,3,147409417,ZIC1,440982,mapped to different gene in human and mouse,LOC440982_fiveUTR,5' UTR,9,91365746.0,Zic1
1,329,cg27201382,0.444645,11,27720483,BDNF,BDNF,627,conserved gene and region in mouse and human,BDNF_fiveUTR,...,11,27720483,BDNF,627,conserved gene and region in mouse and human,BDNF_fiveUTR,Promoter (<=1kb),2,109676257.0,Bdnf
2,144,cg11904056,0.341791,5,93583204,NR2F1-AS1,NR2F1,441094,mapped to different gene in human and mouse,NR2F1-AS1_Intron,...,5,93585317,NR2F1,441094,mapped to different gene in human and mouse,NR2F1-AS1_Exon,Promoter (<=1kb),13,78198288.0,Nr2f1
3,44,cg03820088,0.337644,7,23522229,TRA2A,TRA2A,29896,conserved gene but different region in human a...,TRA2A_fiveUTR,...,7,23522229,TRA2A,29896,conserved gene but different region in human a...,TRA2A_fiveUTR,5' UTR,6,49252905.0,Tra2a
4,58,cg04998737,0.336275,5,77645403,OTP,OTP,23440,conserved gene and region in mouse and human,OTP_Promoter,...,5,77645452,OTP,23440,conserved gene and region in mouse and human,OTP_Promoter,Promoter (5-6kb),13,94869014.0,Otp


In [12]:
# Print colnames
print(Clock1_2_3_overlap_100.columns)

Index(['index_x', 'var_x', 'beta_clock1', 'CHR_x', 'bp_hg38_x', 'Gene',
       'Gene.hg19_x', 'ENTREZID_x', 'conservationInMouse_x', 'GeneRegionID_x',
       'annotation_x', 'CHR_mm10_x', 'bp_mm10_x', 'Gene_mm10_x', 'index_y',
       'var_y', 'beta_clock2', 'CHR_y', 'bp_hg38_y', 'Gene.hg19_y',
       'ENTREZID_y', 'conservationInMouse_y', 'GeneRegionID_y', 'annotation_y',
       'CHR_mm10_y', 'bp_mm10_y', 'Gene_mm10_y', 'index', 'var', 'beta_clock3',
       'CHR', 'bp_hg38', 'Gene.hg19', 'ENTREZID', 'conservationInMouse',
       'GeneRegionID', 'annotation', 'CHR_mm10', 'bp_mm10', 'Gene_mm10'],
      dtype='object')


#### 3.3. Create lists with the CpGs identified by the clocks

In [12]:
# Make a list of the CpGs (var) for each clock 

Clock1_CpG_list = Clock_1['var'].tolist()
Clock2_CpG_list = Clock_2['var'].tolist()
Clock3_CpG_list = Clock_3['var'].tolist()
Clock1_2_3_overlap_CpG_list = Clock_1_2_3_overlap['var_x'].tolist()
Clock2_3_overlap_CpG_list = Clock_2_3_overlap['var_x'].tolist()

Clock1_100_CpG_list = Clock1_100['var'].tolist()
Clock2_100_CpG_list = Clock2_100['var'].tolist()
Clock3_100_CpG_list = Clock3_100['var'].tolist()
Clock1_2_3_overlap_100_CpG_list = Clock1_2_3_overlap_100['var_x'].tolist()
Clock2_3_overlap_100_CpG_list = Clock2_3_overlap_100['var_x'].tolist()

Clock1_200_CpG_list = Clock1_200['var'].tolist()
Clock2_200_CpG_list = Clock2_200['var'].tolist()
Clock3_200_CpG_list = Clock3_200['var'].tolist()
Clock1_2_3_overlap_200_CpG_list = Clock1_2_3_overlap_200['var_x'].tolist()
Clock2_3_overlap_200_CpG_list = Clock2_3_overlap_200['var_x'].tolist()


In [13]:
print(Clock1_100_CpG_list)

['cg11728741', 'cg24352905', 'cg08938156', 'cg13058338', 'cg26067250', 'cg27201382', 'cg00593462', 'cg09227056', 'cg11904056', 'cg03820088', 'cg04998737', 'cg08074329', 'cg17143801', 'cg04368876', 'cg05401971', 'cg16960327', 'cg09710440', 'cg03605454', 'cg16599143', 'cg22661206', 'cg18821963', 'cg15682828', 'cg23087015', 'cg12651099', 'cg12156848', 'cg09622673', 'cg22264409', 'cg11084334', 'cg15340018', 'cg23985931', 'cg08676748', 'cg03370924', 'cg01429475', 'cg12373771', 'cg25943666', 'cg00249943', 'cg14381350', 'cg26844246', 'cg23076498', 'cg16867657', 'cg25019875', 'cg09497746', 'cg04097289', 'cg02532525', 'cg17267107', 'cg05475048', 'cg10125423', 'cg12841266', 'cg15437942', 'cg25468783', 'cg12830057', 'cg04211240', 'cg15719056', 'cg01009870', 'cg18194685', 'cg26012482', 'cg05575054', 'cg27553955', 'cg03564272', 'cg00513357', 'cg24357465', 'cg19981759', 'cg26512254', 'cg03942000', 'cg03185524', 'cg14384416', 'cg24952195', 'cg07499079', 'cg10040493', 'cg25217825', 'cg16570119', 'cg18

#### 3.4. Filter the full and human methylation datasets by the CpG lists & save resulting datasets

In [15]:
# Filtering the full methylation data by the FULL Clocks

Clock1_methylation_data_complete = complete_methylation_data[complete_methylation_data['CpG'].isin(Clock1_CpG_list)]
Clock1_methylation_data_complete = Clock1_methylation_data_complete.set_index('CpG').loc[Clock1_CpG_list].reset_index()
Clock1_methylation_data_human = human_methylation_data[human_methylation_data['CpG'].isin(Clock1_CpG_list)]
Clock1_methylation_data_human = Clock1_methylation_data_human.set_index('CpG').loc[Clock1_CpG_list].reset_index()

Clock2_methylation_data_complete = complete_methylation_data[complete_methylation_data['CpG'].isin(Clock2_CpG_list)]
Clock2_methylation_data_complete = Clock2_methylation_data_complete.set_index('CpG').loc[Clock2_CpG_list].reset_index()
Clock2_methylation_data_human = human_methylation_data[human_methylation_data['CpG'].isin(Clock2_CpG_list)]
Clock2_methylation_data_human = Clock2_methylation_data_human.set_index('CpG').loc[Clock2_CpG_list].reset_index()

Clock3_methylation_data_complete = complete_methylation_data[complete_methylation_data['CpG'].isin(Clock3_CpG_list)]
Clock3_methylation_data_complete = Clock3_methylation_data_complete.set_index('CpG').loc[Clock3_CpG_list].reset_index()
Clock3_methylation_data_human = human_methylation_data[human_methylation_data['CpG'].isin(Clock3_CpG_list)]
Clock3_methylation_data_human = Clock3_methylation_data_human.set_index('CpG').loc[Clock3_CpG_list].reset_index()

Clock1_2_3_overlap_methylation_data_complete = complete_methylation_data[complete_methylation_data['CpG'].isin(Clock1_2_3_overlap_CpG_list)]
Clock1_2_3_overlap_methylation_data_complete = Clock1_2_3_overlap_methylation_data_complete.set_index('CpG').loc[Clock1_2_3_overlap_CpG_list].reset_index()
Clock1_2_3_overlap_methylation_data_human = human_methylation_data[human_methylation_data['CpG'].isin(Clock1_2_3_overlap_CpG_list)]
Clock1_2_3_overlap_methylation_data_human = Clock1_2_3_overlap_methylation_data_human.set_index('CpG').loc[Clock1_2_3_overlap_CpG_list].reset_index()

Clock2_3_overlap_methylation_data_complete = complete_methylation_data[complete_methylation_data['CpG'].isin(Clock2_3_overlap_CpG_list)]
Clock2_3_overlap_methylation_data_complete = Clock2_3_overlap_methylation_data_complete.set_index('CpG').loc[Clock2_3_overlap_CpG_list].reset_index()
Clock2_3_overlap_methylation_data_human = human_methylation_data[human_methylation_data['CpG'].isin(Clock2_3_overlap_CpG_list)]
Clock2_3_overlap_methylation_data_human = Clock2_3_overlap_methylation_data_human.set_index('CpG').loc[Clock2_3_overlap_CpG_list].reset_index()

In [16]:
# # Filtering the full methylation data by the Clocks top 100 CpG's

Clock1_100_methylation_data_complete = complete_methylation_data[complete_methylation_data['CpG'].isin(Clock1_100_CpG_list)]
Clock1_100_methylation_data_complete = Clock1_100_methylation_data_complete.set_index('CpG').loc[Clock1_100_CpG_list].reset_index()
Clock1_100_methylation_data_human = human_methylation_data[human_methylation_data['CpG'].isin(Clock1_100_CpG_list)]
Clock1_100_methylation_data_human = Clock1_100_methylation_data_human.set_index('CpG').loc[Clock1_100_CpG_list].reset_index()

Clock2_100_methylation_data_complete = complete_methylation_data[complete_methylation_data['CpG'].isin(Clock2_100_CpG_list)]
Clock2_100_methylation_data_complete = Clock2_100_methylation_data_complete.set_index('CpG').loc[Clock2_100_CpG_list].reset_index()
Clock2_100_methylation_data_human = human_methylation_data[human_methylation_data['CpG'].isin(Clock2_100_CpG_list)]
Clock2_100_methylation_data_human = Clock2_100_methylation_data_human.set_index('CpG').loc[Clock2_100_CpG_list].reset_index()

Clock3_100_methylation_data_complete = complete_methylation_data[complete_methylation_data['CpG'].isin(Clock3_100_CpG_list)]
Clock3_100_methylation_data_complete = Clock3_100_methylation_data_complete.set_index('CpG').loc[Clock3_100_CpG_list].reset_index()
Clock3_100_methylation_data_human = human_methylation_data[human_methylation_data['CpG'].isin(Clock3_100_CpG_list)]
Clock3_100_methylation_data_human = Clock3_100_methylation_data_human.set_index('CpG').loc[Clock3_100_CpG_list].reset_index()

Clock1_2_3_overlap_100_methylation_data_complete = complete_methylation_data[complete_methylation_data['CpG'].isin(Clock1_2_3_overlap_100_CpG_list)]
Clock1_2_3_overlap_100_methylation_data_complete = Clock1_2_3_overlap_100_methylation_data_complete.set_index('CpG').loc[Clock1_2_3_overlap_100_CpG_list].reset_index()
Clock1_2_3_overlap_100_methylation_data_human = human_methylation_data[human_methylation_data['CpG'].isin(Clock1_2_3_overlap_100_CpG_list)]
Clock1_2_3_overlap_100_methylation_data_human = Clock1_2_3_overlap_100_methylation_data_human.set_index('CpG').loc[Clock1_2_3_overlap_100_CpG_list].reset_index()

Clock2_3_overlap_100_methylation_data_complete = complete_methylation_data[complete_methylation_data['CpG'].isin(Clock2_3_overlap_100_CpG_list)]
Clock2_3_overlap_100_methylation_data_complete = Clock2_3_overlap_100_methylation_data_complete.set_index('CpG').loc[Clock2_3_overlap_100_CpG_list].reset_index()
Clock2_3_overlap_100_methylation_data_human = human_methylation_data[human_methylation_data['CpG'].isin(Clock2_3_overlap_100_CpG_list)]
Clock2_3_overlap_100_methylation_data_human = Clock2_3_overlap_100_methylation_data_human.set_index('CpG').loc[Clock2_3_overlap_100_CpG_list].reset_index()

In [17]:
# Filtering the full methylation data by the Clock's top 200 CpGs

Clock1_200_methylation_data_complete = complete_methylation_data[complete_methylation_data['CpG'].isin(Clock1_200_CpG_list)]
Clock1_200_methylation_data_complete = Clock1_200_methylation_data_complete.set_index('CpG').loc[Clock1_200_CpG_list].reset_index()
Clock1_200_methylation_data_human = human_methylation_data[human_methylation_data['CpG'].isin(Clock1_200_CpG_list)]
Clock1_200_methylation_data_human = Clock1_200_methylation_data_human.set_index('CpG').loc[Clock1_200_CpG_list].reset_index()

Clock2_200_methylation_data_complete = complete_methylation_data[complete_methylation_data['CpG'].isin(Clock2_200_CpG_list)]
Clock2_200_methylation_data_complete = Clock2_200_methylation_data_complete.set_index('CpG').loc[Clock2_200_CpG_list].reset_index()
Clock2_200_methylation_data_human = human_methylation_data[human_methylation_data['CpG'].isin(Clock2_200_CpG_list)]
Clock2_200_methylation_data_human = Clock2_200_methylation_data_human.set_index('CpG').loc[Clock2_200_CpG_list].reset_index()

Clock3_200_methylation_data_complete = complete_methylation_data[complete_methylation_data['CpG'].isin(Clock3_200_CpG_list)]
Clock3_200_methylation_data_complete = Clock3_200_methylation_data_complete.set_index('CpG').loc[Clock3_200_CpG_list].reset_index()
Clock3_200_methylation_data_human = human_methylation_data[human_methylation_data['CpG'].isin(Clock3_200_CpG_list)]
Clock3_200_methylation_data_human = Clock3_200_methylation_data_human.set_index('CpG').loc[Clock3_200_CpG_list].reset_index()

Clock1_2_3_overlap_200_methylation_data_complete = complete_methylation_data[complete_methylation_data['CpG'].isin(Clock1_2_3_overlap_200_CpG_list)]
Clock1_2_3_overlap_200_methylation_data_complete = Clock1_2_3_overlap_200_methylation_data_complete.set_index('CpG').loc[Clock1_2_3_overlap_200_CpG_list].reset_index()
Clock1_2_3_overlap_200_methylation_data_human = human_methylation_data[human_methylation_data['CpG'].isin(Clock1_2_3_overlap_200_CpG_list)]
Clock1_2_3_overlap_200_methylation_data_human = Clock1_2_3_overlap_200_methylation_data_human.set_index('CpG').loc[Clock1_2_3_overlap_200_CpG_list].reset_index()

Clock2_3_overlap_200_methylation_data_complete = complete_methylation_data[complete_methylation_data['CpG'].isin(Clock2_3_overlap_200_CpG_list)]
Clock2_3_overlap_200_methylation_data_complete = Clock2_3_overlap_200_methylation_data_complete.set_index('CpG').loc[Clock2_3_overlap_200_CpG_list].reset_index()
Clock2_3_overlap_200_methylation_data_human = human_methylation_data[human_methylation_data['CpG'].isin(Clock2_3_overlap_200_CpG_list)] 
Clock2_3_overlap_200_methylation_data_human = Clock2_3_overlap_200_methylation_data_human.set_index('CpG').loc[Clock2_3_overlap_200_CpG_list].reset_index()

In [21]:
# Save the data - Complete Clocks
Clock1_methylation_data_complete.to_csv('./Clocks Methylation Data/Clock1_methylation_data_complete.csv', index=False)
Clock1_methylation_data_human.to_csv('./Clocks Methylation Data/Clock1_methylation_data_human.csv', index=False)

Clock2_methylation_data_complete.to_csv('./Clocks Methylation Data/Clock2_methylation_data_complete.csv', index=False)
Clock2_methylation_data_human.to_csv('./Clocks Methylation Data/Clock2_methylation_data_human.csv', index=False)

Clock3_methylation_data_complete.to_csv('./Clocks Methylation Data/Clock3_methylation_data_complete.csv', index=False)
Clock3_methylation_data_human.to_csv('./Clocks Methylation Data/Clock3_methylation_data_human.csv', index=False)

Clock1_2_3_overlap_methylation_data_complete.to_csv('./Clocks Methylation Data/Clock1_2_3_overlap_methylation_data_complete.csv', index=False)
Clock1_2_3_overlap_methylation_data_human.to_csv('./Clocks Methylation Data/Clock1_2_3_overlap_methylation_data_human.csv', index=False)

Clock2_3_overlap_methylation_data_complete.to_csv('./Clocks Methylation Data/Clock2_3_overlap_methylation_data_complete.csv', index=False)
Clock2_3_overlap_methylation_data_human.to_csv('./Clocks Methylation Data/Clock2_3_overlap_methylation_data_human.csv', index=False)       

In [22]:
# Save the data - Top 100 CpGs
Clock1_100_methylation_data_complete.to_csv('./Clocks Methylation Data/Clock1_100_methylation_data_complete.csv', index=False)
Clock1_100_methylation_data_human.to_csv('./Clocks Methylation Data/Clock1_100_methylation_data_human.csv', index=False)

Clock2_100_methylation_data_complete.to_csv('./Clocks Methylation Data/Clock2_100_methylation_data_complete.csv', index=False)
Clock2_100_methylation_data_human.to_csv('./Clocks Methylation Data/Clock2_100_methylation_data_human.csv', index=False)

Clock3_100_methylation_data_complete.to_csv('./Clocks Methylation Data/Clock3_100_methylation_data_complete.csv', index=False)
Clock3_100_methylation_data_human.to_csv('./Clocks Methylation Data/Clock3_100_methylation_data_human.csv', index=False)

Clock1_2_3_overlap_100_methylation_data_complete.to_csv('./Clocks Methylation Data/Clock1_2_3_overlap_100_methylation_data_complete.csv', index=False)
Clock1_2_3_overlap_100_methylation_data_human.to_csv('./Clocks Methylation Data/Clock1_2_3_overlap_100_methylation_data_human.csv', index=False)

Clock2_3_overlap_100_methylation_data_complete.to_csv('./Clocks Methylation Data/Clock2_3_overlap_100_methylation_data_complete.csv', index=False)
Clock2_3_overlap_100_methylation_data_human.to_csv('./Clocks Methylation Data/Clock2_3_overlap_100_methylation_data_human.csv', index=False)

In [23]:
# Save the data - Top 200 CpGs
Clock1_200_methylation_data_complete.to_csv('./Clocks Methylation Data/Clock1_200_methylation_data_complete.csv', index=False)
Clock1_200_methylation_data_human.to_csv('./Clocks Methylation Data/Clock1_200_methylation_data_human.csv', index=False)

Clock2_200_methylation_data_complete.to_csv('./Clocks Methylation Data/Clock2_200_methylation_data_complete.csv', index=False)
Clock2_200_methylation_data_human.to_csv('./Clocks Methylation Data/Clock2_200_methylation_data_human.csv', index=False)

Clock3_200_methylation_data_complete.to_csv('./Clocks Methylation Data/Clock3_200_methylation_data_complete.csv', index=False)
Clock3_200_methylation_data_human.to_csv('./Clocks Methylation Data/Clock3_200_methylation_data_human.csv', index=False)

Clock1_2_3_overlap_200_methylation_data_complete.to_csv('./Clocks Methylation Data/Clock1_2_3_overlap_200_methylation_data_complete.csv', index=False)
Clock1_2_3_overlap_200_methylation_data_human.to_csv('./Clocks Methylation Data/Clock1_2_3_overlap_200_methylation_data_human.csv', index=False)

Clock2_3_overlap_200_methylation_data_complete.to_csv('./Clocks Methylation Data/Clock2_3_overlap_200_methylation_data_complete.csv', index=False)
Clock2_3_overlap_200_methylation_data_human.to_csv('./Clocks Methylation Data/Clock2_3_overlap_200_methylation_data_human.csv', index=False)

### 4. Adding metadata information about each sample of the Human dataset 
(Note: there is a more up to data metadata file with the complete mammal metadata, including human)

#### 4.1. Scrapping metadata from GEO

In [178]:
# Making a list with the sample names (without the first column - CpG names)   
sample_names = methylation_data.columns[1:]

In [180]:
# Scraping metadata for all human samples in the dataset

metadata_human_samples = []

for page_num in range(1, 3):
    try:
        url = f'https://www.ncbi.nlm.nih.gov/geo/browse/?view=samples&display=1000&series=184211&page={page_num}'
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        html_text = soup.get_text()

        for sample_name in sample_names:
            search_term = sample_name.split('X')[1]
            pattern = r'\b(GSM\d+)\s+Sample\d+\.Human\.{}\b'.format(re.escape(search_term))
            match = re.search(pattern, html_text)
            
            if match:
                geo_id = match.group(1)
                url = f'https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc={geo_id}'
                response = requests.get(url)
                response.raise_for_status()
                soup = BeautifulSoup(response.content, 'html.parser')
                
                description_tag = soup.find('td', string=lambda text: text and 'Description' in text)
                description = description_tag.find_next('td').text.strip()
                characteristics_tag = soup.find('td', string=lambda text: text and 'Characteristics' in text)
                characteristics = characteristics_tag.find_next('td').text.strip()
                
                tissue = description.split('tissue=')[1].split(' ')[0]
                age = characteristics.split('age: ')[1].split('confidenceinageestimate')[0].strip()
                sex = characteristics.split('Sex: ')[1].split('\n')[0]
                
                metadata_human_samples.append([sample_name, geo_id, tissue, age, sex])
    
    except Exception as e:
        print(f"Error processing page {page_num}: {e}")

# Check if metadata list is empty
if not metadata_human_samples:
    print("No samples found.")
else:
    # Create a pandas DataFrame from the metadata list
    human_metadata_df = pd.DataFrame(metadata_human_samples, columns=['Sample', 'GEO_ID', 'Tissue', 'Age', 'Sex'])
    # Print the DataFrame
    # print(metadata_df)

# Check for missing samples
found_samples = human_metadata_df['Sample'].tolist() if not human_metadata_df.empty else []
missing_samples = [sample_name for sample_name in sample_names if sample_name not in found_samples]

# Print counts of samples found and not found
print("Total samples:", len(sample_names))
print("Samples found:", len(found_samples))
print("Samples not found:", len(missing_samples))
print('List of Samples not found:', missing_samples)

Total samples: 661
Samples found: 661
Samples not found: 0
List of Samples not found: []


In [23]:
# Sneak peek of the metadata
human_metadata_df.head(5)

,Sample,GEO_ID,Tissue,Age,Sex,Age Group
0,X203203210100_R01C01,GSM5580349,Adipose,50,Male,30-60
1,X203203210100_R02C01,GSM5580350,Adipose,44,Female,30-60
2,X203203210100_R03C01,GSM5580351,Adipose,52,Female,30-60
3,X203203210100_R04C01,GSM5580352,Adipose,62,Female,60-90
4,X203203210100_R05C01,GSM5580353,Adipose,65,Male,60-90


In [24]:
# Shape of the human metadata dataframe
human_metadata_df.shape

(661, 6)

Check the number of unique tissues present in the metadata

In [25]:
# Check the number of unique tissues
unique_tissues = human_metadata_df['Tissue'].nunique()
print(f'Unique tissue types in the metadata: {unique_tissues}')

Unique tissue types in the metadata: 11


In [26]:
# How many samples are there for each tissue type?
tissue_counts = human_metadata_df['Tissue'].value_counts()
tissue_counts


Tissue
Lung          113
Heart          97
Kidney         97
Liver          97
Spleen         62
Adipose        57
Muscle         57
Blood          32
LymphNode      27
BoneMarrow     20
Pituitary       2
Name: count, dtype: int64

Check the range of ages and decide if I can split the ages into groups (possibly less than 30 and more than 30)

In [27]:
# Check the min ad max age
min_age = human_metadata_df['Age'].min()
max_age = human_metadata_df['Age'].max()
print(f'Minimum age: {min_age}')
print(f'Maximum age: {max_age}')

Minimum age: 26
Maximum age: 91


In [29]:
# Proportion of samples for which ages is less than 30
age_less_than_30 = human_metadata_df
age_less_than_30['Age'] = pd.to_numeric(age_less_than_30['Age'], errors='coerce')
age_less_than_30 = age_less_than_30[age_less_than_30['Age'] < 30]
proportion_age_less_than_30 = len(age_less_than_30) / len(human_metadata_df)
print(f'Number of samples for which ages is less than 30: {len(age_less_than_30)}')
print(f'Proportion of samples with age less than 30: {proportion_age_less_than_30}')

Number of samples for which ages is less than 30: 13
Proportion of samples with age less than 30: 0.019667170953101363


In [30]:
# Separate the data into age groups (0-30, 30-60, 60-90, 90+)
age_groups = []

for age in human_metadata_df['Age']:
    if age < 30:
        age_group = '0-30'
    elif age < 60:
        age_group = '30-60'
    elif age < 90:
        age_group = '60-90'
    else:
        age_group = '90+'
    age_groups.append(age_group)

# Add the age groups to the metadata DataFrame
human_metadata_df['Age Group'] = age_groups

# Check the number of samples in each age group
age_group_counts = human_metadata_df
age_group_counts = age_group_counts.groupby('Age Group').size()
print(age_group_counts)

Age Group
0-30      13
30-60    356
60-90    289
90+        3
dtype: int64


In [32]:
# Add age group to the metadata
human_metadata_df['Age Group'] = age_groups
human_metadata_df.head(5)

,Sample,GEO_ID,Tissue,Age,Sex,Age Group
0,X203203210100_R01C01,GSM5580349,Adipose,50,Male,30-60
1,X203203210100_R02C01,GSM5580350,Adipose,44,Female,30-60
2,X203203210100_R03C01,GSM5580351,Adipose,52,Female,30-60
3,X203203210100_R04C01,GSM5580352,Adipose,62,Female,60-90
4,X203203210100_R05C01,GSM5580353,Adipose,65,Male,60-90


In [33]:
# Save the metadata to a CSV file
human_metadata_df.to_csv('human_metadata_184211.csv', index=False)

In [37]:
# Import back if needed (so we don't have to scrape again)
human_metadata_df = pd.read_csv('human_metadata_184211.csv')

In [34]:
human_methylation_data.head(5)

,CpG,X203203210100_R01C01,X203203210100_R02C01,X203203210100_R03C01,X203203210100_R04C01,X203203210100_R05C01,X203203210100_R06C01,X203203210100_R01C02,X203203210100_R02C02,X203203210100_R03C02,...,X203203210013_R03C01,X203203210013_R04C01,X203203210013_R05C01,X203203210013_R06C01,X203203210013_R01C02,X203203210013_R02C02,X203203210013_R03C02,X203203210013_R04C02,X203203210013_R05C02,X203203210013_R06C02
0,cg00000165,0.101505,0.109506,0.141691,0.099335,0.119663,0.133722,0.160619,0.103879,0.113632,...,0.134059,0.111548,0.127344,0.084265,0.112711,0.098072,0.096010,0.117737,0.093164,0.184909
1,cg00001209,0.874657,0.899091,0.856218,0.896215,0.881654,0.919595,0.897742,0.895401,0.888192,...,0.857043,0.826831,0.866019,0.831738,0.852618,0.906398,0.884948,0.846944,0.924727,0.873856
2,cg00001364,0.809139,0.841104,0.768823,0.812510,0.767511,0.855345,0.825716,0.876175,0.845755,...,0.848691,0.865939,0.903710,0.920990,0.860893,0.895960,0.894021,0.896620,0.906478,0.891837
3,cg00001582,0.040719,0.042991,0.041190,0.047361,0.043020,0.043740,0.044908,0.037882,0.046859,...,0.035247,0.034831,0.039376,0.040148,0.040499,0.039100,0.040631,0.043228,0.045163,0.040598
4,cg00002920,0.415627,0.562619,0.492849,0.537996,0.646229,0.324161,0.240706,0.492182,0.404285,...,0.329255,0.315051,0.321974,0.303632,0.351651,0.405474,0.495430,0.309546,0.355513,0.082500


In [35]:
human_metadata_df.head(5)

,Sample,GEO_ID,Tissue,Age,Sex,Age Group
0,X203203210100_R01C01,GSM5580349,Adipose,50,Male,30-60
1,X203203210100_R02C01,GSM5580350,Adipose,44,Female,30-60
2,X203203210100_R03C01,GSM5580351,Adipose,52,Female,30-60
3,X203203210100_R04C01,GSM5580352,Adipose,62,Female,60-90
4,X203203210100_R05C01,GSM5580353,Adipose,65,Male,60-90


Heatmaps of methylation grouped by tissue were done in R and are available in folder "Heatmaps" withit the current directory.

### 7. Statistical Analysis

In [222]:
# Perform t-test for each CpG site between age groups - All data

# Import the necessary libraries
import pandas as pd
from scipy import stats
import statsmodels.stats.multitest as multi

# Load the merged data if needed
human_merged_data = pd.read_csv('human_merged_data_184211.csv')

# Create a list to store the results
t_test_results = []

# Iterate over each CpG site
for cpg_site in human_merged_data.columns[6:]:
    # Perform t-test between age groups
    t_test_result = stats.ttest_ind(
        human_merged_data[human_merged_data['Age Group'] == '0-30'][cpg_site],
        human_merged_data[human_merged_data['Age Group'] == '30-60'][cpg_site]
    )
    # Store the results
    t_test_results.append([cpg_site, t_test_result.pvalue])

# Create a DataFrame from the results
t_test_results_df = pd.DataFrame(t_test_results, columns=['CpG', 'P-Value'])

# Perform multiple testing correction
t_test_results_df['FDR'] = multi.fdrcorrection(t_test_results_df['P-Value'])[1]

# Filter significant CpG sites
significant_cpg_sites = t_test_results_df[t_test_results_df['FDR'] < 0.05]

# Check the number of significant CpG sites
print(f'Number of significant CpG sites: {len(significant_cpg_sites)}')

# Save the results to a CSV file    
t_test_results_df.to_csv('t_test_results_184211.csv', index=False)

# Save the significant CpG sites to a CSV file
significant_cpg_sites.to_csv('significant_cpg_sites_184211.csv', index=False)


Number of significant CpG sites: 103


# TO DO:
# find the average of each Cpg in the full dataset
# find the average of each Cpg in the top 100 CpGs dataset
# compare the averages
